In [1]:
import gym
import numpy as np
import torch
import torchkit.pytorch_utils as ptu
import torchsde
from torch.nn import functional as F
import random as rnd
import copy as cp
# import environments
import envs.pomdp
import pdb
# import recurrent model-free RL (separate architecture)
from policies.models.policy_rnn import ModelFreeOffPolicy_Separate_RNN as Policy_RNN
from policies.models.policy_rnn_shared import ModelFreeOffPolicy_Shared_RNN as Policy_Shared_RNN
from policies.models.policy_mlp import ModelFreeOffPolicy_MLP as Policy_MLP
from tqdm import tqdm
# import the replay buffer
from buffers.seq_replay_buffer_vanilla import SeqReplayBuffer
from buffers.simple_replay_buffer import SimpleReplayBuffer 
from utils import helpers as utl
from typing import Sequence
from read_ini import read_ini
conf =read_ini("C:/Users/alexander.vasilyev/pomdp-baselines-main/configfile.ini")

C:\Users\alexander.vasilyev\Anaconda3\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(
C:\Users\alexander.vasilyev\pomdp-baselines-main\utils\logger.py:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import OrderedDict, Set
C:\Users\alexander.vasilyev\Anaconda3\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


## Build a POMDP environment: Pendulum-V (only observe the velocity)

In [2]:
cuda_id = 0  # -1 if using cpu
ptu.set_gpu_mode(torch.cuda.is_available() and cuda_id >= 0, cuda_id)

env = gym.make(conf["env_name"])
max_trajectory_len = env._max_episode_steps
act_dim = env.action_space.shape[0]
obs_dim = env.observation_space.shape[0]

shared = False
markov = False

if markov:
    agent = Policy_MLP(
        obs_dim=obs_dim,
        action_dim=act_dim,
        algo_name=conf["algo_name"],
        dqn_layers=[128, 128],
        policy_layers=[128, 128],
        lr=3e-4,
        gamma=0.99,
        tau=5e-3,
    ).to(ptu.device)
    encoder="Nan"
else:
    if shared:
        agent = Policy_Shared_RNN(
            obs_dim=obs_dim,
            action_dim=act_dim,
            encoder=conf["encoder"],
            algo_name=conf["algo_name"],
            action_embedding_size=int(conf["action_embedding_size"]),
            observ_embedding_size=int(conf["observ_embedding_size"]),
            reward_embedding_size=int(conf["reward_embedding_size"]),
            rnn_hidden_size=int(conf["hidden_size"]),
            dqn_layers=[128, 128],
            policy_layers=[128, 128],
            lr=float(conf["lr"]),
            gamma=0.9,
            tau=0.005,
            embed=True,
        ).to(ptu.device)
    else: 
        agent = Policy_RNN(
            obs_dim=obs_dim,
            action_dim=act_dim,
            encoder=conf["encoder"],
            algo_name=conf["algo_name"],
            action_embedding_size=int(conf["action_embedding_size"]),
            observ_embedding_size=int(conf["observ_embedding_size"]),
            reward_embedding_size=int(conf["reward_embedding_size"]),
            rnn_hidden_size=int(conf["hidden_size"]),
            dqn_layers=[128, 128],
            policy_layers=[128, 128],
            lr=float(conf["lr"]),
            gamma=0.9,
            tau=0.005,
            radii=40,
            embed=True,
            activation = conf["activation"],
        ).to(ptu.device)
    
print(agent)
lr=float(conf["lr"])
encoder=conf["encoder"]
num_updates_per_iter = int(conf["num_updates_per_iter"])  # training frequency
sampled_seq_len = int(conf["sampled_seq_len"])  # context length
buffer_size = int(float(conf["buffer_size"]))
batch_size = int(conf["batch_size"])
dropout_rate=float(conf["dropout_rate"])
num_iters = int(conf["num_iters"])
num_init_rollouts_pool = int(conf["num_init_rollouts_pool"])
num_rollouts_per_iter = int(conf["num_rollouts_per_iter"])
total_rollouts = num_init_rollouts_pool + num_iters * num_rollouts_per_iter
n_env_steps_total = max_trajectory_len * total_rollouts
_n_env_steps_total = 0
print("total env episodes", total_rollouts, "total env steps", n_env_steps_total)


ModelFreeOffPolicy_Separate_RNN(
  (critic): Critic_RNN(
    (observ_embedder): FeatureExtractor(
      (fc): Linear(in_features=1, out_features=32, bias=True)
    )
    (action_embedder): FeatureExtractor(
      (fc): Linear(in_features=1, out_features=8, bias=True)
    )
    (reward_embedder): FeatureExtractor(
      (fc): Linear(in_features=1, out_features=8, bias=True)
    )
    (rnn): NeuralCDE(
      (func): CDEFunc(
        (linear0): Linear(in_features=72, out_features=72, bias=True)
        (linear1): Linear(in_features=72, out_features=72, bias=True)
        (linear2): Linear(in_features=72, out_features=3528, bias=True)
      )
      (initial): Linear(in_features=49, out_features=72, bias=True)
      (readout): Linear(in_features=72, out_features=72, bias=True)
    )
    (current_shortcut_embedder): FeatureExtractor(
      (fc): Linear(in_features=2, out_features=48, bias=True)
    )
    (qf1): FlattenMlp(
      (fc0): Linear(in_features=120, out_features=128, bias=True)
   

## Build a recurent model-free RL agent: separate architecture, `lstm` encoder, `oar` policy input space, `td3` RL algorithm (context length set later)

## Define other training parameters such as context length and training frequency

## Define key functions: collect rollouts and policy update

In [3]:
def create_ncde_row(obs, next_obs, prev_action, action, prev_reward, reward, steps,init):
    
    if init:
        obs_row= obs
        rew_row = prev_reward
        act_row = prev_action
    else:
        obs_row=torch.cat((obs, next_obs),0)
        rew_row=torch.cat((prev_reward, reward),0)
        act_row=torch.cat((prev_action, action),0)
 
    if shared: 
        obs_row=agent.observ_embedder(obs_row)
        rew_row=agent.reward_embedder(rew_row)
        act_row=agent.action_embedder(act_row)
    else: 
        obs_row=agent.actor.observ_embedder(obs_row)
        rew_row=agent.actor.reward_embedder(rew_row)
        act_row=agent.actor.action_embedder(act_row)
    
    if init:
        time_tensor=torch.tensor([[steps]]).to(ptu.device)
    else:
        time_tensor=torch.tensor([[steps],[steps+1]]).to(ptu.device)

    ncde_row=torch.cat((time_tensor,act_row,obs_row,rew_row),1)
    ncde_row=ncde_row[None,:]
    
    return ncde_row



@torch.no_grad()
def collect_rollouts(
    num_rollouts, random_actions=False, deterministic=True, train_mode=True
):
    """collect num_rollouts of trajectories in task and save into policy buffer
    :param
        random_actions: whether to use policy to sample actions, or randomly sample action space
        deterministic: deterministic action selection?
        train_mode: whether to train (stored to buffer) or test
    """
    if not train_mode:
        assert random_actions == False and deterministic == True

    total_steps = 0
    total_rewards = 0.0
    trewards =[]
    for idx in range(num_rollouts):
        steps = 0
        rewards = 0.0
        energy = 0.0
        print(env.reset())
        obs = ptu.from_numpy(env.reset())
        obs = obs.reshape(1, obs.shape[-1])
        done_rollout = False
        init=True
        # get hidden state at timestep=0, None for mlp
        
        if not markov:
            action, reward, internal_state = agent.get_initial_info()

            if encoder == "ncde":
                internal_state= None
                ncde_row= create_ncde_row(obs, obs, action, action, reward, reward, steps,init)
                prev_action= action.clone()
                prev_reward= reward.clone()
                next_obs= obs.clone()
        
        
        if train_mode:
            # temporary storage
            obs_list, act_list, rew_list, next_obs_list, term_list = (
                [],
                [],
                [],
                [],
                [],
            )
                           

        while not done_rollout:
            if markov: 
                action = agent.act(obs=obs, deterministic=deterministic)[0]
            else:
                if encoder == "ncde":
                    (action,_,_,_), internal_state= agent.ncde_act(ncde_row=ncde_row, prev_internal_state=internal_state, obs=obs,  deterministic=deterministic)
                else:
                    (action, _, _, _), internal_state = agent.act(
                        prev_internal_state=internal_state,
                        prev_action=action,
                        reward=reward,
                        obs=obs,
                        deterministic=deterministic,
                    )
            # observe reward and next obs (B=1, dim)
            #pdb.set_trace()
        
            #print(torch.norm(internal_state))
            next_obs, reward, done, info = utl.env_step(env, action.squeeze(dim=0))
            done_rollout = False if ptu.get_numpy(done[0][0]) == 0.0 else True
            init=False
            
            if not markov:
                if encoder == "ncde":
   
                    ncde_row= create_ncde_row(obs, next_obs, prev_action, action, prev_reward, reward, steps,init)
            
            #switch on/off dropouts
            #drop_trigger=rnd.uniform(0,1)
            #if drop_trigger<dropout_rate:
            #    next_obs=cp.deepcopy(obs)
            # update statistics
           
            rewards += reward.item()
            energy += action*action
           
            # early stopping env: such as rmdp, pomdp, generalize tasks. term ignores timeout
            term = (
                False
                if "TimeLimit.truncated" in info or steps >= max_trajectory_len
                else done_rollout
            )

            if train_mode:
                # append tensors to temporary storage
                obs_list.append(obs)  # (1, dim)
                act_list.append(action)  # (1, dim)
                rew_list.append(reward)  # (1, dim)
                term_list.append(term)  # bool
                next_obs_list.append(next_obs)  # (1, dim)
            steps += 1
            # set: obs <- next_obs
            obs = next_obs.clone()
            prev_reward= reward.clone()
            prev_action= action.clone()
        if train_mode:
            # add collected sequence to buffer
            policy_storage.add_episode(
                observations=ptu.get_numpy(torch.cat(obs_list, dim=0)),  # (L, dim)
                actions=ptu.get_numpy(torch.cat(act_list, dim=0)),  # (L, dim)
                rewards=ptu.get_numpy(torch.cat(rew_list, dim=0)),  # (L, dim)
                terminals=np.array(term_list).reshape(-1, 1),  # (L, 1)
                next_observations=ptu.get_numpy(
                    torch.cat(next_obs_list, dim=0)
                ),  # (L, dim)
            )
        print(
            "Mode:",
            "Train" if train_mode else "Test",
            "env_steps",
            steps,
            "total rewards",
            rewards,
            "total energy",
            energy,
        )
        total_steps += steps
        total_rewards += rewards
        trewards.append(rewards)
    if train_mode:
        return total_steps
    else:
        return total_rewards / num_rollouts, np.std(trewards)


def update(num_updates, factor):
    rl_losses_agg = {}
    # print(num_updates)
    for update in tqdm(range(num_updates), leave=True):
        # sample random RL batch: in transitions
        batch = ptu.np_to_pytorch_batch(policy_storage.random_episodes(batch_size))
        # RL update
        
        rl_losses = agent.update(batch, factor)

        for k, v in rl_losses.items():
            if update == 0:  # first iterate - create list
                rl_losses_agg[k] = [v]
            else:  # append values
                rl_losses_agg[k].append(v)
    # statistics
    for k in rl_losses_agg:
        rl_losses_agg[k] = np.mean(rl_losses_agg[k])
    return rl_losses_agg

## Train and Evaluate the agent: only costs < 20 min

In [ ]:
policy_storage = SeqReplayBuffer(
    max_replay_buffer_size=buffer_size,
    observation_dim=obs_dim,
    action_dim=act_dim,
    sampled_seq_len=sampled_seq_len,
    sample_weight_baseline=0.0,
)

env_steps = collect_rollouts(
    num_rollouts=num_init_rollouts_pool, random_actions=False, train_mode=True
)
_n_env_steps_total += env_steps

# evaluation parameters
last_eval_num_iters = 10
log_interval = 5
eval_num_rollouts = 10
learning_curve = {
    "x": [],
    "y": [],
    "z": [],
}
epoch=0
lambda_pat = 0.65

while _n_env_steps_total < n_env_steps_total:

    env_steps = collect_rollouts(num_rollouts=num_rollouts_per_iter, train_mode=True)
    _n_env_steps_total += env_steps

    #train_stats = update(int(num_updates_per_iter * env_steps))
    factor= lambda_pat **(epoch )
    #train_stats = update(int(num_updates_per_iter * env_steps))
    train_stats = update(25, lr)
    
    epoch += 1
    current_num_iters = _n_env_steps_total // (
        num_rollouts_per_iter * max_trajectory_len
    )
    if (
        current_num_iters != last_eval_num_iters
        and current_num_iters % log_interval == 0
    ):
        last_eval_num_iters = current_num_iters
        average_returns, std_returns = collect_rollouts(
            num_rollouts=eval_num_rollouts,
            train_mode=False,
            random_actions=False,
            deterministic=True,
        )
        learning_curve["x"].append(_n_env_steps_total)
        learning_curve["y"].append(average_returns)
        learning_curve["z"].append(std_returns)
        print(_n_env_steps_total, average_returns)

buffer RAM usage: 0.02 GB
[0.39930123]
angle
tensor(0.1133)
angle
tensor(0.1140)
angle
tensor(0.1146)
angle
tensor(0.1147)
angle
tensor(0.1266)
angle
tensor(0.1272)
angle
tensor(0.1277)
angle
tensor(0.1280)
angle
tensor(0.1258)
angle
tensor(0.1268)
angle
tensor(0.1275)
angle
tensor(0.1278)
angle
tensor(0.1339)
angle
tensor(0.1341)
angle
tensor(0.1340)
angle
tensor(0.1334)
angle
tensor(0.1438)
angle
tensor(0.1427)
angle
tensor(0.1407)
angle
tensor(0.1380)
angle
tensor(0.1525)
angle
tensor(0.1491)
angle
tensor(0.1456)
angle
tensor(0.1414)
angle
tensor(0.1563)
angle
tensor(0.1510)
angle
tensor(0.1460)
angle
tensor(0.1401)
angle
tensor(0.1536)
angle
tensor(0.1470)
angle
tensor(0.1410)
angle
tensor(0.1341)
angle
tensor(0.1472)
angle
tensor(0.1398)
angle
tensor(0.1341)
angle
tensor(0.1269)
angle
tensor(0.1380)
angle
tensor(0.1314)
angle
tensor(0.1264)
angle
tensor(0.1209)
angle
tensor(0.1297)
angle
tensor(0.1247)
angle
tensor(0.1201)
angle
tensor(0.1159)
angle
tensor(0.1229)
angle
tensor(0.1

C:\Users\alexander.vasilyev\Anaconda3\lib\site-packages\torch\nn\functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor(0.1363)
angle
tensor(0.1347)
angle
tensor(0.1333)
angle
tensor(0.1309)
angle
tensor(0.1191)
angle
tensor(0.1166)
angle
tensor(0.1144)
angle
tensor(0.1122)
angle
tensor(0.1027)
angle
tensor(0.1005)
angle
tensor(0.0987)
angle
tensor(0.0968)
angle
tensor(0.0961)
angle
tensor(0.0946)
angle
tensor(0.0935)
angle
tensor(0.0921)
angle
tensor(0.0930)
angle
tensor(0.0918)
angle
tensor(0.0906)
angle
tensor(0.0893)
angle
tensor(0.0802)
angle
tensor(0.0788)
angle
tensor(0.0774)
angle
tensor(0.0758)
angle
tensor(0.0801)
angle
tensor(0.0789)
angle
tensor(0.0777)
angle
tensor(0.0763)
angle
tensor(0.0924)
angle
tensor(0.0907)
angle
tensor(0.0891)
angle
tensor(0.0872)
angle
tensor(0.0855)
angle
tensor(0.0835)
angle
tensor(0.0817)
angle
tensor(0.0796)
angle
tensor(0.0857)
angle
tensor(0.0835)
angle
tensor(0.0815)
angle
tensor(0.0793)
angle
tensor(0.0890)
angle
tensor(0.0868)
angle
tensor(0.0846)
angle
tensor(0.0825)
angle
tensor(0.0967)
angle
tensor(0.0942)
angle
tensor(0.0916)
angle
tensor(0.0889

angle
tensor(0.0952)
angle
tensor(0.0933)
angle
tensor(0.0913)
angle
tensor(0.0890)
angle
tensor(0.0832)
angle
tensor(0.0814)
angle
tensor(0.0797)
angle
tensor(0.0779)
angle
tensor(0.0718)
angle
tensor(0.0706)
angle
tensor(0.0693)
angle
tensor(0.0680)
angle
tensor(0.0807)
angle
tensor(0.0790)
angle
tensor(0.0771)
angle
tensor(0.0752)
angle
tensor(0.0958)
angle
tensor(0.0932)
angle
tensor(0.0908)
angle
tensor(0.0881)
angle
tensor(0.0984)
angle
tensor(0.0963)
angle
tensor(0.0942)
angle
tensor(0.0920)
angle
tensor(0.1054)
angle
tensor(0.1029)
angle
tensor(0.1003)
angle
tensor(0.0976)
angle
tensor(0.1114)
angle
tensor(0.1079)
angle
tensor(0.1047)
angle
tensor(0.1010)
angle
tensor(0.1126)
angle
tensor(0.1084)
angle
tensor(0.1047)
angle
tensor(0.1005)
angle
tensor(0.1132)
angle
tensor(0.1085)
angle
tensor(0.1043)
angle
tensor(0.0996)
angle
tensor(0.1125)
angle
tensor(0.1076)
angle
tensor(0.1033)
angle
tensor(0.0987)
angle
tensor(0.1120)
angle
tensor(0.1075)
angle
tensor(0.1038)
angle
tensor(

tensor(0.0563)
angle
tensor(0.0549)
angle
tensor(0.0642)
angle
tensor(0.0628)
angle
tensor(0.0614)
angle
tensor(0.0599)
angle
tensor(0.0748)
angle
tensor(0.0727)
angle
tensor(0.0708)
angle
tensor(0.0685)
angle
tensor(0.0915)
angle
tensor(0.0886)
angle
tensor(0.0860)
angle
tensor(0.0830)
angle
tensor(0.0897)
angle
tensor(0.0871)
angle
tensor(0.0846)
angle
tensor(0.0819)
angle
tensor(0.0926)
angle
tensor(0.0895)
angle
tensor(0.0865)
angle
tensor(0.0833)
angle
tensor(0.0992)
angle
tensor(0.0954)
angle
tensor(0.0919)
angle
tensor(0.0880)
angle
tensor(0.1002)
angle
tensor(0.0960)
angle
tensor(0.0923)
angle
tensor(0.0881)
angle
tensor(0.1024)
angle
tensor(0.0979)
angle
tensor(0.0937)
angle
tensor(0.0890)
angle
tensor(0.1034)
angle
tensor(0.0988)
angle
tensor(0.0951)
angle
tensor(0.0911)
angle
tensor(0.1065)
angle
tensor(0.1024)
angle
tensor(0.0988)
angle
tensor(0.0952)
angle
tensor(0.1115)
angle
tensor(0.1076)
angle
tensor(0.1041)
angle
tensor(0.1007)
angle
tensor(0.1182)
angle
tensor(0.1143

...

tensor(0.1227)
angle
tensor(0.1194)
angle
tensor(0.1159)
angle
tensor(0.1169)
angle
tensor(0.1196)
angle
tensor(0.1226)
angle
tensor(0.1251)
angle
tensor(0.1390)
angle
tensor(0.1427)
angle
tensor(0.1463)
angle
tensor(0.1492)
angle
tensor(0.1579)
angle
tensor(0.1612)
angle
tensor(0.1649)
angle
tensor(0.1669)
angle
tensor(0.1702)
angle
tensor(0.1722)
angle
tensor(0.1736)
angle
tensor(0.1738)
angle
tensor(0.1699)
angle
tensor(0.1694)
angle
tensor(0.1681)
angle
tensor(0.1660)
angle
tensor(0.1575)
angle
tensor(0.1556)
angle
tensor(0.1538)
angle
tensor(0.1515)
angle
tensor(0.1416)
angle
tensor(0.1391)
angle
tensor(0.1367)
angle
tensor(0.1341)
angle
tensor(0.1224)
angle
tensor(0.1202)
angle
tensor(0.1180)
angle
tensor(0.1155)
angle
tensor(0.1098)
angle
tensor(0.1079)
angle
tensor(0.1063)
angle
tensor(0.1045)
angle
tensor(0.1023)
angle
tensor(0.1011)
angle
tensor(0.1000)
angle
tensor(0.0987)
angle
tensor(0.0870)
angle
tensor(0.0854)
angle
tensor(0.0839)
angle
tensor(0.0822)
angle
tensor(0.0954

  0%|          | 0/25 [00:00<?, ?it/s]

angle
tensor(0.1953)
angle
tensor(0.1927)
angle
tensor(0.1904)
angle
tensor(0.1876)
angle
tensor(0.1171)
angle
tensor(0.1169)
angle
tensor(0.1165)
angle
tensor(0.1158)
angle
tensor(0.1166)
angle
tensor(0.1161)
angle
tensor(0.1155)
angle
tensor(0.1145)
angle
tensor(0.1211)
angle
tensor(0.1200)
angle
tensor(0.1187)
angle
tensor(0.1172)
angle
tensor(0.1196)
angle
tensor(0.1178)
angle
tensor(0.1157)
angle
tensor(0.1135)
angle
tensor(0.1173)
angle
tensor(0.1149)
angle
tensor(0.1124)
angle
tensor(0.1099)
angle
tensor(0.1172)
angle
tensor(0.1144)
angle
tensor(0.1117)
angle
tensor(0.1089)
angle
tensor(0.1163)
angle
tensor(0.1133)
angle
tensor(0.1104)
angle
tensor(0.1074)
angle
tensor(0.1127)
angle
tensor(0.1099)
angle
tensor(0.1076)
angle
tensor(0.1051)
angle
tensor(0.1205)
angle
tensor(0.1181)
angle
tensor(0.1159)
angle
tensor(0.1135)
angle
tensor(0.1197)
angle
tensor(0.1174)
angle
tensor(0.1153)
angle
tensor(0.1131)
angle
tensor(0.1204)
angle
tensor(0.1187)
angle
tensor(0.1169)
angle
tensor(

C:\Users\alexander.vasilyev\pomdp-baselines-main\torchkit\pytorch_utils.py:73: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if v.dtype == np.bool:


angle
tensor(0.0906)
angle
tensor(0.0888)
angle
tensor(0.0868)
angle
tensor(0.0895)
angle
tensor(0.0873)
angle
tensor(0.0853)
angle
tensor(0.0831)
angle
tensor(0.0835)
angle
tensor(0.0815)
angle
tensor(0.0797)
angle
tensor(0.0778)
angle
tensor(0.0845)
angle
tensor(0.0827)
angle
tensor(0.0811)
angle
tensor(0.0794)
angle
tensor(0.0812)
angle
tensor(0.0797)
angle
tensor(0.0783)
angle
tensor(0.0767)
angle
tensor(0.0989)
angle
tensor(0.0973)
angle
tensor(0.0957)
angle
tensor(0.0939)
angle
tensor(0.1023)
angle
tensor(0.1003)
angle
tensor(0.0983)
angle
tensor(0.0962)
angle
tensor(0.1019)
angle
tensor(0.0998)
angle
tensor(0.0978)
angle
tensor(0.0956)
angle
tensor(0.1014)
angle
tensor(0.0993)
angle
tensor(0.0974)
angle
tensor(0.0954)
angle
tensor(0.0997)
angle
tensor(0.0979)
angle
tensor(0.0963)
angle
tensor(0.0945)
angle
tensor(0.0975)
angle
tensor(0.0959)
angle
tensor(0.0945)
angle
tensor(0.0929)
angle
tensor(0.1127)
angle
tensor(0.1105)
angle
tensor(0.1084)
angle
tensor(0.1062)
angle
tensor(

## Draw the learning curve

In [ ]:
import matplotlib.pyplot as plt
print(learning_curve)
plt.plot(learning_curve["x"], learning_curve["y"])
plt.fill_between(np.array(learning_curve["x"]), np.array(learning_curve["y"])-np.array(learning_curve["z"]), np.array(learning_curve["y"])+np.array(learning_curve["z"]))
plt.xlabel("env steps")
plt.ylabel("return")
plt.show()

In [ ]:
leaning_curve_ncde_64_rk4 = learning_curve

In [ ]:
learning_curve


In [ ]:
timess=torch.linspace(0, 65-1, 65)

In [ ]:
file1 = open('config.txt', 'w')
file1.write(str(conf))

file1.close()
file2 = open('results.txt', 'w')
file2.write(str(learning_curve))
file2.close()